In [1]:
pip install yfinance

In [2]:
pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [16]:
pip install pyngrok

SyntaxError: invalid syntax (<ipython-input-16-c8b6cd76cc61>, line 1)

In [27]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date, datetime
import datetime as dt
from dateutil.relativedelta import relativedelta

import dash
from dash import dcc, html, dash_table, Dash
from dash import Input, Output

import plotly.graph_objects as go

from flask import Flask, request

from pyngrok import ngrok, conf

from google.colab import files

from threading import Thread

In [5]:
#Pull price of symbol
def get_price(ticker):
  tick = yf.Ticker(ticker)
  temp = temp = tick.history(period='1d')
  price = temp.iloc[0,0]
  price = round(price, 2)
  return price

#Pull list of contract expirations for symbol
def get_expirations(ticker):
  tick = yf.Ticker(ticker)
  exps = tick.options
  return exps

#Pull historic price data and calculate vol
def get_volatility(ticker):
  # Fetch historical price data
  price_data = tick.history(period='5y')

  # Calculate the annualized standard deviation (volatility)
  daily_returns = price_data["Close"].pct_change()
  annualized_volatility = daily_returns.std() * np.sqrt(252)

  annualized_volatility = round(annualized_volatility, 4)

  return annualized_volatility

#Pull risk free rate (10 year treasury yield)
def get_risk_free_rate():
  tick = '^TNX'
  tick = yf.Ticker(tick)
  temp = tick.history(period='5d')
  rate = temp.iloc[3,3]
  rate = rate / 100
  rate = round(rate, 4)
  return rate

#Calculate time to maturity given a contract expiration date
def get_time_to_expiration(expiration):
  today = datetime.now()
  exp = datetime.strptime(expiration, '%Y-%m-%d')
  t = exp - today
  return t

In [6]:
#Calculate delta & gamma for an option
  # S = Underlying Symbol Price
  # K = Strike
  # T = Time to Expiration
  # r = Risk Free Rate
  # sigma = Underlying Symbol Vol
  # option_type = Call or Put

######### Not used but helpful for reference ###############

def calc_delta(S, K, T, r, sigma, option_type='call'):
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

  if option_type == 'call':
        delta = norm.cdf(d1)
  elif option_type == 'put':
        delta = norm.cdf(d1) - 1

  return delta

def calc_gamma(S, K, T, r, sigma):
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  gamma = np.exp(-r * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
  return gamma

In [7]:
symbol = 'SPY'

In [56]:
#Pull & Aggregate Option Date -

#Set Yfinance ticker equal to SPY
tick = yf.Ticker(symbol)

#Pull expirations
expirations = tick.options

#Move to pandas dataframe
exps = pd.DataFrame(expirations, columns=['expiration_dates'])

#Calc days to expiration and format
exps['time_to_expiration'] = exps['expiration_dates'].apply(get_time_to_expiration)
exps['time_to_expiration'] = exps['time_to_expiration'].dt.days.astype('int16') / 365

#Get assumptions for delta/gamma calc
S = get_price(symbol)
print()
r = get_risk_free_rate()
sigma = get_volatility(symbol)

#Aggregate data for all expirations & strikes

#Assumptions
print(f"Symbol: {symbol}\nPrice: {S}\nRisk Free Rate: {r}\nVolatility: {sigma}")
print("-----------------------------------------")

#Create emptry dataframeas
df_calls = pd.DataFrame()
df_puts = pd.DataFrame()

#Loop through each expiration date in dataframe
for i in range(len(exps)):

  #Setting independent variable to expiration date ran in loop
  exp_date = exps['expiration_dates'][i]

  #Iteration tracker
  print(f"{i + 1}/{len(exps)}: {exp_date}")

  #Create empty dataframes for calls & puts
  #Resets dataframes from last loop
  new_calls = pd.DataFrame()
  new_puts = pd.DataFrame()

  #Pull in calls & puts data for chosen expiration date
  new_calls = tick.option_chain(exp_date).calls
  new_puts = tick.option_chain(exp_date).puts

  #Assigning dataframe to new calls & puts chain
  new_calls = pd.DataFrame(new_calls)
  new_puts = pd.DataFrame(new_puts)

  new_calls['exp_date'] = exps['expiration_dates'][i]
  new_puts['exp_date'] = exps['expiration_dates'][i]

  #Assign option type
  new_calls['type'] = 'call'
  new_puts['type'] = 'put'

  #Safeguard against using outdated data
  T = exps['time_to_expiration'][i]
  if T < 0:
    T = 0
  else:
    T = T

  dte = T * 365

  new_calls['dte'] = dte
  new_puts['dte'] = dte

  ### DELTA ###

  #Calculate delta for options in dataframes using vectorization
  new_calls['delta'] = norm.cdf((np.log(S / new_calls['strike']) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
  new_puts['delta'] = norm.cdf((np.log(S / new_puts['strike']) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))

  #Round the deltas
  new_calls['delta'] = new_calls['delta'].round(2)
  new_puts['delta'] = new_puts['delta'].round(2)-1

  #Adjust deltas using open interest as weight
  new_calls['weighted_delta'] = new_calls['openInterest'] * new_calls['delta']
  new_puts['weighted_delta'] = new_puts['openInterest'] * new_puts['delta']

  ### GAMMA ###

  #Calculate gamma for options in dataframes using vectorization
  new_calls['gamma'] = (np.exp(-r * T) * norm.pdf((np.log(S / new_calls['strike']) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))) ) / (new_calls['strike'] * sigma * np.sqrt(T))
  new_puts['gamma'] = (np.exp(-r * T) * norm.pdf((np.log(S / new_puts['strike']) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))) ) / (new_puts['strike'] * sigma * np.sqrt(T))

  #Round the gammas
  new_calls['gamma'] = new_calls['gamma'].round(2)
  new_puts['gamma'] = new_puts['gamma'].round(2)*-1

  #Adjust gammas using open interest as weight
  new_calls['weighted_gamma'] = new_calls['openInterest'] * new_calls['gamma']
  new_puts['weighted_gamma'] = new_puts['openInterest'] * new_puts['gamma']

  #Concat new data to existing dataframes
  df_calls = pd.concat([df_calls, new_calls])
  df_puts = pd.concat([df_puts, new_puts])

#Get the current price
current_price = round(S, 0)

#Filter the DataFrame for strikes within 100 points of the current price
strike_range = 100

filtered_calls = df_calls[(df_calls['strike'] >= current_price - strike_range) & (df_calls['strike'] <= current_price + strike_range)]
filtered_puts = df_puts[(df_puts['strike'] >= current_price - strike_range) & (df_puts['strike'] <= current_price + strike_range)]

df = pd.concat([filtered_calls, filtered_puts])

df['strike'] = df['strike'].astype(int)

df = df[df['openInterest'] > 100]

print('\nCOMPLETE')


Symbol: SPY
Price: 521.86
Risk Free Rate: 0.0416
Volatility: 0.1737
-----------------------------------------
1/32: 2025-04-08
2/32: 2025-04-09
3/32: 2025-04-10
4/32: 2025-04-11
5/32: 2025-04-14
6/32: 2025-04-15
7/32: 2025-04-16
8/32: 2025-04-17
9/32: 2025-04-25
10/32: 2025-04-30
11/32: 2025-05-02
12/32: 2025-05-09
13/32: 2025-05-16
14/32: 2025-05-23
15/32: 2025-05-30
16/32: 2025-06-20
17/32: 2025-06-30
18/32: 2025-07-18
19/32: 2025-07-31
20/32: 2025-08-15
21/32: 2025-08-29
22/32: 2025-09-19
23/32: 2025-09-30
24/32: 2025-12-19
25/32: 2025-12-31
26/32: 2026-01-16
27/32: 2026-03-20
28/32: 2026-03-31
29/32: 2026-06-18
30/32: 2026-12-18
31/32: 2027-01-15
32/32: 2027-12-17

COMPLETE


In [9]:
# Save the dataframe to a CSV file
df.to_csv('spy_option_data.csv', index=False)

# Download the CSV file

files.download('spy_option_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
# Create the Dash app
app = dash.Dash(__name__)

# Fill and process the original dataframe for delta and gamma analysis
graph_df = df.fillna(0)
graph_df = graph_df[graph_df['weighted_delta'] != 0]
graph_df = graph_df[graph_df['weighted_gamma'] != 0]

# Group by strike and expiration date for delta and gamma analysis
delta_by_strike = graph_df.groupby('strike')['weighted_delta'].sum()
delta_by_exp = graph_df.groupby('exp_date')['weighted_delta'].sum()

# Separate calls and puts for gamma calculations
df_calls = graph_df[graph_df['type'] == 'call'].groupby(['strike', 'exp_date'])['weighted_gamma'].sum()
df_puts = graph_df[graph_df['type'] == 'put'].groupby(['strike', 'exp_date'])['weighted_gamma'].sum()

# Today's date
today = dt.date.today()

# Add time horizons
gamma_signals = {
    'Short-term': graph_df[graph_df['dte'] <= 7]['weighted_gamma'].sum(),
    'Mid-term': graph_df[(graph_df['dte'] > 7) & (graph_df['dte'] <= 30)]['weighted_gamma'].sum(),
    'Long-term': graph_df[graph_df['dte'] > 30]['weighted_gamma'].sum(),
}

short_term_call_gamma = graph_df[(graph_df['type'] == 'call') & (graph_df['dte'] <= 7)]['weighted_gamma'].sum()
short_term_put_gamma = graph_df[(graph_df['type'] == 'put') & (graph_df['dte'] <= 7)]['weighted_gamma'].sum()
short_term_net_gamma = short_term_call_gamma + short_term_put_gamma

mid_term_call_gamma = graph_df[(graph_df['type'] == 'call') & (graph_df['dte'] > 7) & (graph_df['dte'] <= 30)]['weighted_gamma'].sum()
mid_term_put_gamma = graph_df[(graph_df['type'] == 'put') & (graph_df['dte'] > 7) & (graph_df['dte'] <= 30)]['weighted_gamma'].sum()
mid_term_net_gamma = mid_term_call_gamma + mid_term_put_gamma

long_term_call_gamma = graph_df[(graph_df['type'] == 'call') & (graph_df['dte'] > 30)]['weighted_gamma'].sum()
long_term_put_gamma = graph_df[(graph_df['type'] == 'put') & (graph_df['dte'] > 30)]['weighted_gamma'].sum()
long_term_net_gamma = long_term_call_gamma + long_term_put_gamma

# Classify as Bullish or Bearish
gamma_signals_classified = {key: ('Bullish' if value > 0 else 'Bearish') for key, value in gamma_signals.items()}

# Signal Display with 3-Column Layout
signal_display = html.Div([
    html.H2("Market Sentiment Signals", style={'text-align': 'center'}),
    html.H3(f"SPY Price: ${S:.2f}", style={'text-align': 'center'}),

    # Grid Container for Columns
    html.Div([
        # Short-term Column
        html.Div([
            html.H4("Short-term", style={'text-align': 'center', 'margin-bottom': '10px'}),
            html.Div(f"Total Call Gamma: {short_term_call_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Total Put Gamma: {short_term_put_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Net Gamma: {short_term_net_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Signal: {gamma_signals_classified['Short-term']}",
                     style={'text-align': 'center',
                            'color': 'green' if gamma_signals['Short-term'] > 0 else 'red',
                            'font-weight': 'bold'})
        ], style={'flex': '1', 'border': '1px solid #ccc', 'padding': '10px', 'margin': '5px', 'border-radius': '10px'}),

        # Mid-term Column
        html.Div([
            html.H4("Mid-term", style={'text-align': 'center', 'margin-bottom': '10px'}),
            html.Div(f"Total Call Gamma: {mid_term_call_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Total Put Gamma: {mid_term_put_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Net Gamma: {mid_term_net_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Signal: {gamma_signals_classified['Mid-term']}",
                     style={'text-align': 'center',
                            'color': 'green' if gamma_signals['Mid-term'] > 0 else 'red',
                            'font-weight': 'bold'})
        ], style={'flex': '1', 'border': '1px solid #ccc', 'padding': '10px', 'margin': '5px', 'border-radius': '10px'}),

        # Long-term Column
        html.Div([
            html.H4("Long-term", style={'text-align': 'center', 'margin-bottom': '10px'}),
            html.Div(f"Total Call Gamma: {long_term_call_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Total Put Gamma: {long_term_put_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Net Gamma: {long_term_net_gamma:,.2f}", style={'text-align': 'center'}),
            html.Div(f"Signal: {gamma_signals_classified['Long-term']}",
                     style={'text-align': 'center',
                            'color': 'green' if gamma_signals['Long-term'] > 0 else 'red',
                            'font-weight': 'bold'})
        ], style={'flex': '1', 'border': '1px solid #ccc', 'padding': '10px', 'margin': '5px', 'border-radius': '10px'}),
    ], style={'display': 'flex', 'justify-content': 'space-around', 'margin': '20px'}),
], style={'border': '1px solid #ccc', 'padding': '20px', 'border-radius': '10px', 'margin': '20px'})

# Calculate net gamma by summing call and put gamma
df_net = df_calls.add(df_puts, fill_value=0)
df_net = df_net.reset_index()

# Calculate the top 20 gamma strikes and top 10 expirations based on absolute total gamma
top_20_gamma_strikes = df_net.groupby('strike')['weighted_gamma'].apply(lambda x: abs(x).sum()).nlargest(20).index
top_10_gamma_exp = df_net.groupby('exp_date')['weighted_gamma'].apply(lambda x: abs(x).sum()).nlargest(10).index

# Get filtered calls and puts data based on the top 20 gamma strikes
df_calls_filtered = df_calls[df_calls.index.get_level_values('strike').isin(top_20_gamma_strikes)]
df_puts_filtered = df_puts[df_puts.index.get_level_values('strike').isin(top_20_gamma_strikes)]

# Reset index to avoid ambiguity
df_net_reset = df_net.reset_index()
df_calls_reset = df_calls.reset_index()
df_puts_reset = df_puts.reset_index()

# Delta by strike
delta_by_strike_fig = go.Figure(data=[go.Bar(
  x=delta_by_strike.index,
  y=delta_by_strike.values,
  marker_color=['green' if x > 0 else 'red' for x in delta_by_strike]
)])
delta_by_strike_fig.update_layout(
  title='Delta by Strike',
  title_font=dict(size=20, weight='bold'),
  xaxis_title='Strike Price',
  yaxis_title='Delta'
)

# Delta by expiration
delta_by_exp_fig = go.Figure(data=[go.Bar(
  x=delta_by_exp.index,
  y=delta_by_exp.values,
  marker_color=['green' if x > 0 else 'red' for x in delta_by_exp]
)])
delta_by_exp_fig.update_layout(
  title='Delta by Expiration',
  title_font=dict(size=20, weight='bold'),
  xaxis_title='Expiration Date',
  yaxis_title='Delta'
)

# Gamma by strike
gamma_by_strike_fig = go.Figure()

# Calls
gamma_by_strike_fig.add_trace(go.Bar(
    x=df_calls_reset[df_calls_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum().index,
    y=df_calls_reset[df_calls_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum()['weighted_gamma'].values,
    name='Calls',
    marker_color='green'
))

# Puts
gamma_by_strike_fig.add_trace(go.Bar(
    x=df_puts_reset[df_puts_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum().index,
    y=df_puts_reset[df_puts_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum()['weighted_gamma'].values,
    name='Puts',
    marker_color='red'
))

# Net Gamma
gamma_by_strike_fig.add_trace(go.Bar(
    x=df_net_reset[df_net_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum().index,
    y=df_net_reset[df_net_reset['strike'].isin(top_20_gamma_strikes)].groupby('strike').sum()['weighted_gamma'].values,
    name='Net',
    marker_color='blue'
))

gamma_by_strike_fig.update_layout(
  title='Gamma by Strike (Top 20)',
  title_font=dict(size=20, weight='bold'),
  xaxis_title='Strike Price',
  yaxis_title='Gamma',
  barmode='group'
)

# Gamma by expiration
gamma_by_exp_fig = go.Figure()

gamma_by_exp_fig.add_trace(go.Bar(
x=df_calls_reset[df_calls_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum().index,
y=df_calls_reset[df_calls_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum()['weighted_gamma'].values,
name='Calls',
marker_color='green'
))
gamma_by_exp_fig.add_trace(go.Bar(
x=df_puts_reset[df_puts_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum().index,
  y=df_puts_reset[df_puts_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum()['weighted_gamma'].values,
  name='Puts',
  marker_color='red'
))
gamma_by_exp_fig.add_trace(go.Bar(
  x=df_net_reset[df_net_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum().index,
  y=df_net_reset[df_net_reset['exp_date'].isin(top_10_gamma_exp)].groupby('exp_date').sum()['weighted_gamma'].values,
  name='Net',
  marker_color='blue'
))
gamma_by_exp_fig.update_layout(
  title='Gamma by Expiration (Top 10)',
  title_font=dict(size=20, weight='bold'),
  xaxis_title='Expiration Date',
  yaxis_title='Gamma',
  barmode='group'
)

# Calculate the absolute value of gamma and get the top 50 gamma contracts
graph_df['abs_weighted_gamma'] = graph_df['weighted_gamma'].abs()
top_50_abs_gamma = graph_df.nlargest(50, 'abs_weighted_gamma')

# Scatter plot: Gamma by Strike vs Expiration Date
scatter_plot = go.Figure()

# Add calls (green)
scatter_plot.add_trace(go.Scatter(
    x=top_50_abs_gamma[top_50_abs_gamma['type'] == 'call']['exp_date'],
    y=top_50_abs_gamma[top_50_abs_gamma['type'] == 'call']['strike'],
    mode='markers',
    marker=dict(
        size=top_50_abs_gamma[top_50_abs_gamma['type'] == 'call']['abs_weighted_gamma'] * 0.05,  # Scaling by gamma
        color='green',  # Green for calls
        opacity=0.7
    ),
    name='Calls'
))

# Add puts (red)
scatter_plot.add_trace(go.Scatter(
    x=top_50_abs_gamma[top_50_abs_gamma['type'] == 'put']['exp_date'],
    y=top_50_abs_gamma[top_50_abs_gamma['type'] == 'put']['strike'],
    mode='markers',
    marker=dict(
        size=top_50_abs_gamma[top_50_abs_gamma['type'] == 'put']['abs_weighted_gamma'] * 0.05,  # Scaling by gamma
        color='red',  # Red for puts
        opacity=0.7
    ),
    name='Puts'
))

scatter_plot.update_layout(
    title='Top 50 Gamma Contracts by Strike vs Expiration Date',
    title_font=dict(size=20, weight='bold'),
    xaxis_title='Expiration Date',
    yaxis_title='Strike Price',
    showlegend=True
)

# Sort calls and puts by weighted gamma and select the top 5 for each, including the 'type' column
top_5_calls = df_calls_reset.sort_values('weighted_gamma', ascending=False).head(5)
top_5_puts = df_puts_reset.sort_values('weighted_gamma', ascending=True).head(5)

top_5_calls = round(top_5_calls, 2)
top_5_puts = round(top_5_puts, 2)

# Add the 'type' column explicitly
top_5_calls['type'] = 'call'
top_5_puts['type'] = 'put'

# Combine the top 5 calls and puts into one DataFrame for the table
top_5_combined = pd.concat([top_5_calls, top_5_puts])

# Create the table data
table_data = [
    {"type": row['type'], "strike": row['strike'], "exp_date": row['exp_date'], "weighted_gamma": row['weighted_gamma']}
    for idx, row in top_5_combined.iterrows()
]

# Add a table showing the top 5 calls and puts by gamma
table = dash_table.DataTable(

    id='top-5-table',
    columns=[
        {"name": "Type", "id": "type"},
        {"name": "Strike", "id": "strike"},
        {"name": "Expiration Date", "id": "exp_date"},
        {"name": "Gamma", "id": "weighted_gamma"}
    ],
    data=table_data,  # Use the table data
    style_table={'height': '350px', 'overflowY': 'auto'},  # Table styling
    style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
    style_cell={'textAlign': 'center', 'padding': '10px'},
)

# Add a text blurb
text_blurb = html.Div([
    html.H3("Data Analysis on SPY's option market", style={'text-align': 'left'}),
    html.P("This page breaks down the total amounts of delta and gamma found in SPY's option market. "
           "Gamma is a measure of the rate of change in an option's delta relative to changes in the price of the underlying asset. "
           "Graphs look at the aggregated amounts based on strike, expiration, and type of option.",
           style={'text-align': 'left'})
])

table_title = html.Div([
    html.H3("Top 5 Largest Gamma Contracts (Calls & Puts)", style={'text-align': 'left'}),
])

text_blurb2 = html.Div([
    html.H3("Understanding Dealer Hedging Dynamics", style={'text-align': 'left'}),
    html.P(
        "Dealer hedging plays a key role in market movements. When options market makers sell calls, they buy the underlying stock as its price rises and sell as it falls. "
        "Conversely, when selling puts, they sell the underlying as its price drops and buy as it rises. "
        "This dynamic amplifies price movements around key strike levels where gamma exposure is highest, potentially leading to sharp market reactions.",
        style={'text-align': 'left'}
    )
])

### Layout ###
app.layout = html.Div([
  html.H1("Options Dashboard", style={'text-align': 'center'}),

  text_blurb,

  text_blurb2,

  signal_display,

  # Delta graphs
  dcc.Graph(figure=delta_by_strike_fig),
  dcc.Graph(figure=delta_by_exp_fig),

  # Gamma graphs
  dcc.Graph(figure=gamma_by_strike_fig),
  dcc.Graph(figure=gamma_by_exp_fig),

  # Scatter plot (Top 50 Absolute Gamma)
  dcc.Graph(figure=scatter_plot),

  table_title,
  table

])



In [54]:
#Run Dash
app.run(host="0.0.0.0", port=8050, debug=False, use_reloader=False)

#Setup NGrok Token
ngrok.set_auth_token('2vSHBb1qTDpcxJDgzWD7dkhAZnM_3SiqJMN3duprjJLaKJcY1')

#Open a tunnel to your target port
public_url = ngrok.connect(8050)
print(f"Public URL: {public_url}")

<IPython.core.display.Javascript object>

Public URL: NgrokTunnel: "https://5a98-35-224-48-19.ngrok-free.app" -> "http://localhost:8050"


In [47]:
ngrok.kill()